# Adding the test data to our database

In [31]:
import sqlite3

In [32]:
conn = sqlite3.connect("../database/db.sql")
cur = conn.cursor()

In [33]:
import pandas as pd

df = pd.read_csv("../../assets/test_questions.csv")

In [34]:
df.head()

,question,answer1,answer2,answer3,answer4,correct_answers,points
0,Was ist ein großes Risiko des Klimawandels für...,Erhöhte Belastung durch hitzebedingte Krankheiten,Unterbrechung der Gesundheitsdienste durch ext...,Verbesserte Luftqualität,Sinkende Krankheitsraten,"[""Erhöhte Belastung durch hitzebedingte Krankh...",3
1,Welche Bevölkerungsgruppe ist am meisten vom K...,Junge Erwachsene,Kinder,Ältere Menschen,Gesunde Menschen mittleren Alters,"[""Kinder"", ""Ältere Menschen""]",2
2,"Wie können Einzelpersonen dazu beitragen, die ...",Öffentliche Verkehrsmittel nutzen,Weniger Fleisch konsumieren,Klimaanlagen intensiver nutzen,Mehr Auto fahren,"[""Öffentliche Verkehrsmittel nutzen"", ""Weniger...",2
3,Welches Gesundheitsproblem steht in direktem Z...,Herzkrankheiten,Atemwegserkrankungen,Erhöhte Inzidenz von Infektionskrankheiten,Fettleibigkeit,"[""Atemwegserkrankungen"", ""Erhöhte Inzidenz von...",1
4,Welches der folgenden Ziele ist ein Nachhaltig...,Bezahlbare und saubere Energie,Kein Hunger,Maximaler Ressourcenverbrauch,Unbegrenztes Wirtschaftswachstum,"[""Bezahlbare und saubere Energie"", ""Kein Hunger""]",3


In [35]:
# Function to clear the Question and Answer tables
def clear_tables():
    cur.execute("DELETE FROM Answer")
    cur.execute("DELETE FROM Question")
    conn.commit()
clear_tables()


In [36]:
import csv
import uuid
import ast

# Function to insert data from CSV into the database
def insert_data_from_csv(csv_file):
    with open(csv_file, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        
        for row in reader:
            # generate a unique ID for the question
            question_id = str(uuid.uuid4())
            question_text = row['question']
            points = int(row['points'])
            print(points)


            # insert the question into the Question table
            cur.execute(f"INSERT INTO Question (id, question_text, points) VALUES ('{question_id}', '{question_text}', {points})")
            
            # extract answers and the correct answers list
            answers = [row['answer1'], row['answer2'], row['answer3'], row['answer4']]
            correct_answers = ast.literal_eval(row['correct_answers'])  # Convert correct_answers string to list
            
            # insert each answer into the Answer table
            for answer in answers:
                answer_id = str(uuid.uuid4())  # generate unique ID for each answer
                
                # determine if the current answer is correct
                is_correct = answer in correct_answers
                
                # Insert the answer into the Answer table
                cur.execute("INSERT INTO Answer (id, question_id, answer, is_correct, explanation) VALUES (?, ?, ?, ?, ?)",
                            (answer_id, question_id, answer, is_correct, ""))  # empty explanation for now
            
        conn.commit()

csv_file = "../../assets/test_questions.csv"
insert_data_from_csv(csv_file)



3
2
2
1
3
5
2
3
4
2
1
3
2
4
3
2
4
2
1
4


In [37]:
# testing how Answer table looks like
cur.execute("""
    SELECT *
    FROM ANSWER
    LIMIT 20
""")
rows = cur.fetchall()

for row in rows:
    print(row)


('92df1ccc-fb5b-49af-a48e-3862127760f6', 'f3449b72-fc18-454d-b7c3-5e1ddac307ab', 'Erhöhte Belastung durch hitzebedingte Krankheiten', 1, '')
('2e0e5975-960b-4e60-83cd-67368084e50e', 'f3449b72-fc18-454d-b7c3-5e1ddac307ab', 'Unterbrechung der Gesundheitsdienste durch extreme Wetterereignisse', 1, '')
('202a2db8-a446-43a8-b75d-5b0f09fa4bb5', 'f3449b72-fc18-454d-b7c3-5e1ddac307ab', 'Verbesserte Luftqualität', 0, '')
('356a8b60-eb5d-4133-abc4-52f71fba0ef2', 'f3449b72-fc18-454d-b7c3-5e1ddac307ab', 'Sinkende Krankheitsraten', 0, '')
('0b5d1a1e-7724-4c44-a855-d8d78fbe74b5', '6454e9f8-92e9-41b7-a57c-ac0a164f5d9b', 'Junge Erwachsene', 0, '')
('80038f14-5c01-41d8-aa0e-fd7254936fad', '6454e9f8-92e9-41b7-a57c-ac0a164f5d9b', 'Kinder', 1, '')
('c538f7f7-4a29-446e-8b49-f642aee95dd5', '6454e9f8-92e9-41b7-a57c-ac0a164f5d9b', 'Ältere Menschen', 1, '')
('3ea44daf-dd8c-4f5e-b801-80d53e4b9de5', '6454e9f8-92e9-41b7-a57c-ac0a164f5d9b', 'Gesunde Menschen mittleren Alters', 0, '')
('396fe896-dbca-40c1-8f84-6f1d

In [38]:
# testing how Question table looks like
cur.execute("""
    SELECT *
    FROM Question
    LIMIT 5
""")
rows = cur.fetchall()

for row in rows:
    print(row)

('f3449b72-fc18-454d-b7c3-5e1ddac307ab', 'Was ist ein großes Risiko des Klimawandels für Gesundheitssysteme?', 3)
('6454e9f8-92e9-41b7-a57c-ac0a164f5d9b', 'Welche Bevölkerungsgruppe ist am meisten vom Klimawandel betroffen?', 2)
('0e1021be-75aa-46cd-bca3-3a1ff16232af', 'Wie können Einzelpersonen dazu beitragen, die globale Erwärmung zu reduzieren?', 2)
('19afe325-757f-4f0e-8a5c-195a8b495356', 'Welches Gesundheitsproblem steht in direktem Zusammenhang mit dem Klimawandel?', 1)
('545f77ae-80ed-4bc7-9dc5-826ca9bcbe3d', 'Welches der folgenden Ziele ist ein Nachhaltigkeitsziel (SDG)?', 3)


In [39]:
# Fetch questions and their associated answers
cur.execute("""
    SELECT q.question_text, a.answer, a.is_correct
    FROM Question q
    JOIN Answer a ON q.id = a.question_id
    LIMIT 12
""")
rows = cur.fetchall()

for row in rows:
    print(row)

conn.close()


('Was ist ein großes Risiko des Klimawandels für Gesundheitssysteme?', 'Erhöhte Belastung durch hitzebedingte Krankheiten', 1)
('Was ist ein großes Risiko des Klimawandels für Gesundheitssysteme?', 'Unterbrechung der Gesundheitsdienste durch extreme Wetterereignisse', 1)
('Was ist ein großes Risiko des Klimawandels für Gesundheitssysteme?', 'Verbesserte Luftqualität', 0)
('Was ist ein großes Risiko des Klimawandels für Gesundheitssysteme?', 'Sinkende Krankheitsraten', 0)
('Welche Bevölkerungsgruppe ist am meisten vom Klimawandel betroffen?', 'Junge Erwachsene', 0)
('Welche Bevölkerungsgruppe ist am meisten vom Klimawandel betroffen?', 'Kinder', 1)
('Welche Bevölkerungsgruppe ist am meisten vom Klimawandel betroffen?', 'Ältere Menschen', 1)
('Welche Bevölkerungsgruppe ist am meisten vom Klimawandel betroffen?', 'Gesunde Menschen mittleren Alters', 0)
('Wie können Einzelpersonen dazu beitragen, die globale Erwärmung zu reduzieren?', 'Öffentliche Verkehrsmittel nutzen', 1)
('Wie können Ei

In [40]:
conn.close()